Retreiving Earthdata products using Python with shapefile

In [ ]:
import geopandas as gpd
import earthaccess
import xarray as xr
import fnmatch
import numpy as np
import os
from shapely.geometry import Point
from shapely.geometry import shape
from shapely import vectorized

In [ ]:
auth = earthaccess.login(strategy= "interactive", persist=True)

earthaccess.login()

In [ ]:
results = earthaccess.search_data(
    short_name= 'SWOT_L2_HR_PIXC_2.0)', # Product name
    bounding_box= (-114, 62, -115, 63), # Coordinates of bounding box
    temporal= ("2024-01", "2025-01"), # Period of time
    count=-1)

search_word = "*_330_045R*"  # Enter granule name, * will search for anything within the phrase

filtered_results = [result for result in results if fnmatch.fnmatch(result.data_links()[0], search_word)]

files = earthaccess.download(filtered_results, "./local_folder_01") 

downloaded_filenames = [os.path.basename(file_path) for file_path in files]

print("Downloaded filenames:")
for name in downloaded_filenames:
    print(name)

In [ ]:
netcdf_path = r"local_folder_01/SWOT_L2_HR_PIXC_027_330_045R_20250125T111918_20250125T111929_PIC2_01.nc"
shapefile_path = r"HydroLAKES_polys_v10_shp/HydroLAKES_polys_v10.shp"

shapefile = gpd.read_file(shapefile_path)
lake_poly = shapefile[shapefile['Hylak_id'] == 363]

lake_geom = lake_poly.geometry.values[0]

ds = xr.open_dataset(netcdf_path, group='pixel_cloud')

lat = ds['latitude'].values
lon = ds['longitude'].values
hgt = ds['height'].values
sig = ds['sig0'].values

mask = vectorized.contains(lake_geom, lon, lat)

filtered_lat = lat[mask]
filtered_lon = lon[mask]
filtered_hgt = hgt[mask]
filtered_sig = sig[mask]

filtered_points = gpd.GeoDataFrame({
    'longitude': filtered_lon,
    'latitude': filtered_lat,
    'height': filtered_hgt,
    'sig0': filtered_sig
}, geometry=[Point(xy) for xy in zip(filtered_lon, filtered_lat)], crs='EPSG:4326')

ds.close()